# GLLVM Simulations

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from gllvm import GLLVM, PoissonGLM, BinomialGLM, GaussianGLM


seed = 123

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# ----------------------------------------------------------
# 1. Generate synthetic data from the ground-truth model
# ----------------------------------------------------------

gllvm0 = GLLVM(latent_dim=1, output_dim=5)
gllvm0.add_glm(PoissonGLM, idx=[0, 1], name="Poisson1")
gllvm0.add_glm(BinomialGLM, idx=[2], name="Binomial1")
gllvm0.add_glm(GaussianGLM, idx=[3, 4], name="Gaussian1")

num_samples = 10_000
z0 = gllvm0.sample_z(num_samples)
y0 = gllvm0.sample(z=z0)

# ----------------------------------------------------------
# 2. Build a fresh model that will be trained with VI
# ----------------------------------------------------------

gllvm = GLLVM(latent_dim=1, output_dim=5)
gllvm.add_glm(PoissonGLM, idx=[0, 1], name="Poisson1")
gllvm.add_glm(BinomialGLM, idx=[2], name="Binomial1")
gllvm.add_glm(GaussianGLM, idx=[3, 4], name="Gaussian1")

# ----------------------------------------------------------
# 3. Build the encoder q(z|y)
# ----------------------------------------------------------

class Encoder(nn.Module):
    def __init__(self, input_dim=5, latent_dim=1, hidden=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
        )
        self.mean = nn.Linear(hidden, latent_dim)
        self.logvar = nn.Linear(hidden, latent_dim)

    def forward(self, y):
        h = self.net(y)
        mu = self.mean(h)
        logvar = self.logvar(h)
        return mu, logvar

encoder = Encoder(input_dim=5, latent_dim=1)
optimizer = optim.Adam(list(gllvm.parameters()) + list(encoder.parameters()), lr=1e-3)

# ----------------------------------------------------------
# 4. VI training loop: Standard VAE
# ----------------------------------------------------------

batch_size = 256
num_epochs = 500

dataset = y0
n = len(dataset)

for epoch in range(num_epochs):
    perm = torch.randperm(n)

    total_elbo = 0.0
    for i in range(0, n, batch_size):
        idx = perm[i:i+batch_size]
        y = dataset[idx]

        # Encode q(z|y)
        mu, logvar = encoder(y)
        std = torch.exp(0.5 * logvar)

        # Reparameterization
        eps = torch.randn_like(std)
        z = mu + eps * std

        # Decoder log-likelihood
        logpy_z = gllvm.log_prob(y, z=z).sum(dim=-1)  # sum over response dims

        # KL(q||p)
        kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1)

        elbo = (logpy_z - kl).mean()
        loss = -elbo

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_elbo += elbo.item()

    print(f"Epoch {epoch+1}: ELBO={total_elbo:.2f}")



Epoch 1: ELBO=-312.20
Epoch 2: ELBO=-295.69
Epoch 3: ELBO=-290.18
Epoch 4: ELBO=-288.21
Epoch 5: ELBO=-288.37
Epoch 6: ELBO=-286.27
Epoch 7: ELBO=-285.98
Epoch 8: ELBO=-284.73
Epoch 9: ELBO=-284.60
Epoch 10: ELBO=-284.14
Epoch 11: ELBO=-284.09
Epoch 12: ELBO=-283.42
Epoch 13: ELBO=-283.15
Epoch 14: ELBO=-281.33
Epoch 15: ELBO=-281.03
Epoch 16: ELBO=-281.40
Epoch 17: ELBO=-281.54
Epoch 18: ELBO=-280.52
Epoch 19: ELBO=-280.40
Epoch 20: ELBO=-279.52
Epoch 21: ELBO=-278.98
Epoch 22: ELBO=-278.01
Epoch 23: ELBO=-279.28
Epoch 24: ELBO=-277.38
Epoch 25: ELBO=-276.97
Epoch 26: ELBO=-276.87
Epoch 27: ELBO=-275.88
Epoch 28: ELBO=-275.29
Epoch 29: ELBO=-273.99
Epoch 30: ELBO=-273.20
Epoch 31: ELBO=-272.88
Epoch 32: ELBO=-272.35
Epoch 33: ELBO=-271.78
Epoch 34: ELBO=-271.50
Epoch 35: ELBO=-270.09
Epoch 36: ELBO=-270.19
Epoch 37: ELBO=-269.08
Epoch 38: ELBO=-269.60
Epoch 39: ELBO=-268.53
Epoch 40: ELBO=-268.71
Epoch 41: ELBO=-268.44
Epoch 42: ELBO=-267.27
Epoch 43: ELBO=-267.37
Epoch 44: ELBO=-267.

In [5]:
gllvm.wz

Parameter containing:
tensor([[ 0.1104, -0.1402, -1.3724, -1.3199, -0.8128]], requires_grad=True)

In [3]:
gllvm0.wz

Parameter containing:
tensor([[ 0.1188, -0.1348, -1.3689, -1.2970, -0.8029]], requires_grad=True)

In [11]:
print("True W_z:\n", gllvm0.wz)
print("Estimated W_z:\n", gllvm.wz)

print("True bias:\n", gllvm0.bias)
print("Estimated bias:\n", gllvm.bias)

print("True scale:\n", gllvm0.scale)
print("Estimated scale:\n", gllvm.scale)


True W_z:
 Parameter containing:
tensor([[-0.7856, -0.1608, -0.7251,  0.5009, -1.0102]], requires_grad=True)
Estimated W_z:
 Parameter containing:
tensor([[ 0.7671,  0.1664,  0.6880, -0.4997,  1.0128]], requires_grad=True)
True bias:
 Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)
Estimated bias:
 Parameter containing:
tensor([ 0.0222,  0.0058,  0.0084,  0.0049, -0.0112], requires_grad=True)
True scale:
 Parameter containing:
tensor([1., 1., 1., 1., 1.], requires_grad=True)
Estimated scale:
 Parameter containing:
tensor([1.0000, 1.0000, 1.0000, 0.9924, 1.0000], requires_grad=True)
